In [1]:
# load package
import numpy as np
import pandas as pd
import pickle
import xlearn as xl
import logging

# set path
import os
default_path = "/Users/francislin/JrML/03_04_What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# all_data.pickle is made from prepare_X.ipynb
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)

In [3]:
df.head()

,cuisine,id,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,...,used_hock,used_rose,used_ciabatta,used_ready,used_regular,used_glass,used_safflower,used_jeera,used_Oil,used_oven
0,greek,10259,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,25693,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,20130,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,22213,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,13162,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y_list=[]
for a in df.cuisine:
    if a not in Y_list and not pd.isnull(a):
        Y_list.append(a)
Y_list
len(Y_list)

['greek',
 'southern_us',
 'filipino',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'brazilian',
 'french',
 'japanese',
 'irish',
 'korean',
 'moroccan',
 'russian']

20

In [5]:
# make 20 Y in df
for a in Y_list:
    df['%s'% (a)]= df['cuisine'].str.contains('%s'% (a),na=False)*1

In [6]:
df[102:105]

,cuisine,id,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,...,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
102,italian,28851,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,greek,4635,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,korean,7782,0,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,1,0,0


In [7]:
df_Y= df[(['id','cuisine']+Y_list)]
df_Y.head()

df_key= df[['id','cuisine']]
emb_k5_all=df_key # prepare mother table for first time in the following loops

,id,cuisine,greek,southern_us,filipino,indian,jamaican,spanish,italian,mexican,...,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
0,10259,greek,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 區分train test
df=df.drop(Y_list,axis=1)
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns

df_Y_Train= df_Y[pd.notnull(df['cuisine'])]

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_hock', 'used_rose', 'used_ciabatta', 'used_ready', 'used_regular',
       'used_glass', 'used_safflower', 'used_jeera', 'used_Oil', 'used_oven'],
      dtype='object', length=902)

In [9]:
fm_model = xl.FMModel(task='binary', init=0.1, epoch=10, k=5, lr=0.1, reg_lambda=0.1, opt='sgd', metric='auc')
fm_model

FMModel(alpha=1, beta=1, block_size=500, epoch=10, fold=1, init=0.1, k=5,
    lambda_1=1, lambda_2=1, lr=0.1, metric='auc', model_type='fm',
    n_jobs=4, nthread=None, opt='sgd', reg_lambda=0.1, stop_window=2,
    task='binary')

In [10]:
import logging
def X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col):
    logging.info('Fit the FM model and derive the embedding values')
    fm_model.fit(X_dummies_train, emb_X_input[Y_col]) #Y_col should be a str
    emb_weight = fm_model.weights[1]
    X_emb = np.matmul(X_dummies.values, emb_weight)
    X_emb = pd.DataFrame(X_emb, columns=[Y_col+'_'+str(i) for i in range(emb_weight.shape[1])])
    return X_emb
    

In [11]:
X_dummies = df.drop(['id','cuisine'],axis=1)
X_dummies_train = dataTrain.drop(['id','cuisine'],axis=1)
for y in Y_list:
    emb_X_input = df_Y_Train[['id', y]]#.reset_index(drop=True)
    Y_col=y
    X_emb= X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col)
    emb_k5_all= pd.concat([emb_k5_all,X_emb], axis=1)
    emb_k5_all.shape
emb_k5_all.head()

(49718, 7)

(49718, 12)

(49718, 17)

(49718, 22)

(49718, 27)

...

(49718, 87)

(49718, 92)

(49718, 97)

(49718, 102)

,id,cuisine,greek_0,greek_1,greek_2,greek_3,greek_4,southern_us_0,southern_us_1,southern_us_2,...,moroccan_0,moroccan_1,moroccan_2,moroccan_3,moroccan_4,russian_0,russian_1,russian_2,russian_3,russian_4
0,10259,greek,1.504321e-07,8.413370e-08,1.370113e-07,2.121038e-07,1.632968e-07,1.543959e-07,2.140319e-07,6.299994e-08,...,-1.130970e-07,-7.015309e-08,-9.339560e-08,-9.050575e-08,-9.213072e-08,-4.951313e-08,2.716038e-08,5.095515e-08,8.442199e-08,-4.981065e-08
1,25693,southern_us,-1.265237e-07,-1.207269e-07,-1.303789e-07,-1.450370e-07,-1.379893e-07,4.497118e-08,-6.007453e-08,-1.016296e-07,...,-3.739040e-08,-3.328418e-08,-5.998014e-08,-5.202901e-08,-2.362114e-08,-8.755257e-09,1.815557e-07,1.930086e-07,2.269317e-07,2.697356e-08
2,20130,filipino,2.376367e-05,8.009876e-06,2.221927e-05,2.332269e-05,1.412222e-05,2.383523e-05,8.072647e-06,2.226723e-05,...,2.384332e-05,8.108578e-06,2.228810e-05,2.341495e-05,1.422062e-05,2.397364e-05,8.376593e-06,2.261572e-05,2.377516e-05,1.434198e-05
3,22213,indian,-3.136633e-08,-4.659835e-08,-5.006742e-08,-4.438030e-08,-3.584083e-08,-3.380205e-08,-6.689425e-08,-1.081901e-07,...,-7.424820e-09,-2.092047e-08,-2.166416e-08,-2.478439e-08,-8.544654e-09,2.082947e-08,2.252624e-07,2.156138e-07,2.244563e-07,6.707796e-08
4,13162,indian,3.496644e-05,1.502816e-04,5.439540e-05,6.885732e-05,6.643060e-05,3.458811e-05,1.500296e-04,5.402166e-05,...,3.518619e-05,1.505639e-04,5.466352e-05,6.915050e-05,6.675429e-05,3.517699e-05,1.507897e-04,5.494560e-05,6.949222e-05,6.670450e-05


In [12]:
# check the global min
emb_k5_all.min().min()

-5.26817855e-06

In [13]:
import pickle
with open('./data/all_data_emb_k5.pickle','wb') as f:
    pickle.dump(emb_k5_all,f)

### embedding one time before loops

In [ ]:
# Just test one time in this cell
emb_X_input = df_Y_Train[['id','greek']]#.reset_index(drop=True)
X_dummies = df.drop(['id','cuisine'],axis=1)
X_dummies_train = dataTrain.drop(['id','cuisine'],axis=1)
Y_col='greek'

In [ ]:
X_emb= X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col)